In [2]:
!pip install MetaTrader5
!pip install yfinance --upgrade --no-cache-dir
!pip install get-all-tickers #https://github.com/shilewenuw/get_all_tickers/tree/master/get_all_tickers
!pip install html5lib

ERROR: Invalid requirement: '#https://github.com/shilewenuw/get_all_tickers/tree/master/get_all_tickers'


# Desafio Final - I2A2 Financial Market

- Devemos montar uma análise do momentum do mercado;

## Requisitos:

- Análise das classificações do ativo, como:
    * Tendência do ativo (LTA/LTB/Lateralizando)
    * Suporte/Resistência
    * Formação de Candle (escolher algumas)
    * Médias móveis (9, 21)
    * Willians %
    * RSI
    * MACD
    * Bollinger Bands
- Utilizar o último mês de dados do ativo

In [3]:
from src.logger import logger



from src.exchanges.bittrex import Bittrex
from src.exchanges.binance import Binance
from src.exchanges.marketdata import marketdata

from src.trader import Trader
from src.messenger import Messenger
from src.database import Database

from utils.utils import get_secrets, get_settings
from src.directory_utilities import get_json_from_file

from src.indicators import Indicators
from datetime import datetime, timedelta
from binance.client import Client
import logging
import pandas as pd
import numpy as np

from src.graphical_analysis import analyzer, generate_graph

In [4]:
secrets = get_secrets()
settings = get_settings()

db = Database()
_trader = Trader(secrets, settings, operator = marketdata)

#_coin_pair = settings.get('coin_pair')
_coin_pair = "AAPL"

* O código foi montado com todos os indicadores concentrados no arquivo "indicadores.py", podendo ser modificado conforme a necessidade;
* A classe marketdata contém o fluxo de download dos dados, usando o yfinance, e uma modificação para encaixar em um formato comum com outras classes de recuperação de dados (como da binance, de criptoativos)
* Ao passarmos o mesmo como argumento do Trader (que é a nossa "classe-mestre"), o mesmo consultará os dados usando a interface de exchanges em comum entre todas (bittrex, binance e marketdata);
* O Trader recupera todos os indicadores na implementação da get_historical_prices, retornando os DFs que utilizamos para interpretação no analyzer;
* O Analyzer, por si só, "monta" as mensagens que precisamos, traduzindo o gráfico em palavras mais simples, recuperando o snapshot que é o objetivo do trabalho.

Acompanhemos:

In [5]:
%%time
df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend = _trader.get_historical_prices(_coin_pair)

[*********************100%***********************]  1 of 1 completed
Wall time: 6.85 s


In [6]:
df2 = analyzer(_coin_pair).translate_graph_fromdf(df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend)

In [7]:
df2[['Datetime','translated_message']]

,Datetime,translated_message
0,2021-01-11 21:00:00,"O ativo AAPL, na data de 11/01/2021, ás 21:00:..."
1,2021-01-12 21:00:00,"O ativo AAPL, na data de 12/01/2021, ás 21:00:..."
2,2021-01-13 21:00:00,"O ativo AAPL, na data de 13/01/2021, ás 21:00:..."
3,2021-01-14 21:00:00,"O ativo AAPL, na data de 14/01/2021, ás 21:00:..."
4,2021-01-18 21:00:00,"O ativo AAPL, na data de 18/01/2021, ás 21:00:..."
...,...,...
248,2022-01-04 21:00:00,"O ativo AAPL, na data de 04/01/2022, ás 21:00:..."
249,2022-01-05 21:00:00,"O ativo AAPL, na data de 05/01/2022, ás 21:00:..."
250,2022-01-06 21:00:00,"O ativo AAPL, na data de 06/01/2022, ás 21:00:..."
251,2022-01-09 21:00:00,"O ativo AAPL, na data de 09/01/2022, ás 21:00:..."


In [8]:
print(df2.iloc[len(df2)].translated_message)

IndexError: single positional indexer is out-of-bounds

In [10]:
generate_graph(df, crossovers)